In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [26]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [27]:
wiki.name

'wikipedia'

In [18]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
loader=WebBaseLoader("https://langchain.com/docs/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024EC9582910>, search_kwargs={})

In [30]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o-mini",temperature=0)


In [20]:
retriever.invoke("Conceptual guide")

[Document(id='c73d8e0f-fa46-4dcc-b6e6-d6bd88a9196d', metadata={'source': 'https://langchain.com/docs/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content="Contributing\u200b\nCheck out the developer's guide for guidelines on contributing and help getting your dev environment set up.Edit this pageWas this page helpful?NextTutorialsArchitectureGuidesTutorialsHow-to guidesConceptual guideIntegrationsAPI referenceEcosystem🦜🛠️ LangSmith🦜🕸️ LangGraphAdditional resourcesVersionsSecurityContributingCommunityTwitterGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2025 LangChain, Inc."),
 Document(id='71db254c-c4e6-45d4-a9ea-484bfdf5e82d', metadata={'source': 'https://langchain.com/docs/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'languag

In [22]:
from langchain.tools.retriever import create_retriever_tool
retrieval_tool=create_retriever_tool(retriever,"langchain search","search for information about langchain . for any question asked about langchain use this tool")

In [23]:
retrieval_tool.name

'langchain search'

In [24]:
#Arxic tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)


In [25]:
arxiv.name

'arxiv'

In [47]:
tools=[wiki,arxiv] 


In [48]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\irfan\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200))]

AGENT

In [37]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [49]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)


AGENT EXECUTER

In [50]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
# agent_executor.invoke({"input":"what is the latest news on langchain"})
agent_executor



AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [52]:
agent_executor.invoke({"input":"tell me about attention is all you need paper"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'Attention is All You Need'}`


Published: 2024-07-22
Title: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models
Authors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervi
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new de
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need paper'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deThe paper "Attention Is All You Need," published in 2017 by researchers at Google, is a seminal work in the field of machine learning, particular

{'input': 'tell me about attention is all you need paper',
 'output': 'The paper "Attention Is All You Need," published in 2017 by researchers at Google, is a seminal work in the field of machine learning, particularly in natural language processing (NLP). It introduced the Transformer model, which relies entirely on attention mechanisms, discarding the recurrent layers used in previous models like LSTMs and GRUs.\n\n### Key Contributions:\n1. **Transformer Architecture**: The paper proposed a novel architecture that uses self-attention mechanisms to process input data. This allows the model to weigh the importance of different words in a sentence, regardless of their position.\n\n2. **Self-Attention Mechanism**: The self-attention mechanism enables the model to consider the entire context of a sentence when encoding a word, which improves understanding and generation of language.\n\n3. **Parallelization**: Unlike RNNs, which process data sequentially, the Transformer model allows for 

In [38]:
!pip install langchainhub


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
!pip install langchain-openai


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
!pip install wikipedia


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
!pip install arxiv


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
